In [1]:
#install Ultralytics

%pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


# Creating train-val split

In [2]:
## importing required libraries
import os
import shutil
import random



In [3]:
%pip install tqdm --upgrade
from tqdm import tqdm

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
#for sorting data
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [5]:


def train_test_split(path,neg_path=None, split = 0.2):


    files = list(set([name[:-4] for name in os.listdir(path)])) 
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)

    
    #images to training folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')
        
    
    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

    


    ### copyting images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    
    print("Completed")



train_test_split('/Users/christinemathews/Downloads/yoloV8/data/') 

  0%|          | 0/53 [00:00<?, ?it/s]

100%|██████████| 13/13 [00:00<00:00, 237.72it/s]

Completed


In [10]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.27 🚀 Python-3.9.6 torch-2.2.1 CPU (Apple M2)
Setup complete ✅ (8 CPUs, 8.0 GB RAM, 140.7/228.3 GB disk)


## Training

In [12]:
!yolo task=detect mode=train model=yolov8s.pt data=dataset.yaml epochs=10 imgsz=640 batch=8 project=/Users/christinemathews/Downloads/yoloV8/training_result/ name=football 

/Users/christinemathews/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Ultralytics YOLOv8.1.27 🚀 Python-3.9.6 torch-2.2.1 CPU (Apple M2)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=dataset.yaml, epochs=10, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/Users/christinemathews/Downloads/yoloV8/training_result/, name=football2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=30

## Inferencing

In [13]:
!yolo task=detect mode=predict model=training_result/football2/weights/best.pt conf=0.65 source=test

/Users/christinemathews/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Ultralytics YOLOv8.1.27 🚀 Python-3.9.6 torch-2.2.1 CPU (Apple M2)
[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

image 1/6 /Users/christinemathews/Downloads/yoloV8/test/22d_f_135_we.jpg: 384x640 12 players, 460.6ms
image 2/6 /Users/christinemathews/Downloads/yoloV8/test/22d_f_185_6ty.jpg: 384x640 15 players, 270.9ms
image 3/6 /Users/christinemathews/Downloads/yoloV8/test/fr_f_100_we.jpg: 384x640 13 players, 1 football, 268.4ms
image 4/6 /Users/christinemathews/Downloads/yoloV8/test/fr_f_70.jpg: 384x640 8 players, 1 football, 264.2ms
image 5/6 /Users/christinemathews/Downloads/yoloV8/test/we_f_115_f

In [17]:
!cp -r /Users/christinemathews/Desktop/Swiss/football-processing-opencv/runs/detect/predict /Users/christinemathews/Downloads/yoloV8/output

### inferencing on video


In [29]:
!yolo task=detect mode=predict model=training_result/football2/weights/best.pt conf=0.77 source=/Users/christinemathews/Downloads/yoloV8/videos

/Users/christinemathews/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Ultralytics YOLOv8.1.27 🚀 Python-3.9.6 torch-2.2.1 CPU (Apple M2)
[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

video 1/2 (frame 1/150) /Users/christinemathews/Downloads/yoloV8/videos/test.mp4: 384x640 13 players, 415.6ms
video 1/2 (frame 2/150) /Users/christinemathews/Downloads/yoloV8/videos/test.mp4: 384x640 13 players, 243.4ms
video 1/2 (frame 3/150) /Users/christinemathews/Downloads/yoloV8/videos/test.mp4: 384x640 13 players, 250.9ms
video 1/2 (frame 4/150) /Users/christinemathews/Downloads/yoloV8/videos/test.mp4: 384x640 13 players, 278.2ms
video 1/2 (frame 5/150) /Users/christinemathews/Down

In [30]:
!cp -r /Users/christinemathews/Desktop/Swiss/football-processing-opencv/runs/detect/predict3 /Users/christinemathews/Downloads/yoloV8/output

In [27]:
!yolo mode=export model=training_result/football2/weights/best.pt 

/Users/christinemathews/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
WARNING ⚠️ 'format' is missing. Using default 'format=torchscript'.
Ultralytics YOLOv8.1.27 🚀 Python-3.9.6 torch-2.2.1 CPU (Apple M2)
[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from 'training_result/football2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (21.5 MB)

TorchScript: starting export with torch 2.2.1...
TorchScript: export success ✅ 5.7s, saved as 'training_result/football2/weights/best.torchscript' (42.8 MB)

Export complete (8.3s)
Results saved to /Users/christinemathews/Downloads/yoloV8/training_result/football2/weights
P